## Q1. Embedding the query

In [45]:
from fastembed import TextEmbedding
import numpy as np

In [46]:
query = 'I just discovered the course. Can I join now?'
model = TextEmbedding("jinaai/jina-embeddings-v2-small-en")
embedding = list(model.embed(query))
np.min(embedding)

np.float64(-0.11726373885183883)

In [47]:
np.linalg.norm(embedding[0])

np.float64(1.0)

## Q2. Cosine similarity with another vector

In [4]:
doc = 'Can I still join the course after the start date?'
embedding_q1 = embedding[0]
embedding_q2 = list(model.embed(doc))[0]
embedding_q1.dot(embedding_q2)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [5]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [6]:
V = []
for d in documents:
    embedding = list(model.embed(d['text']))[0]
    V.append(embedding)
V=np.array(V)

In [7]:
cosine_q3 = V.dot(embedding_q2)
cosine_q3

array([0.81060572, 0.84989067, 0.79913355, 0.68130471, 0.67392881])

In [8]:
int(np.argmax(cosine_q3))

1

## Q4. Ranking by cosine, version two

In [9]:
V = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    embedding = list(model.embed(full_text))[0]
    V.append(embedding)
V=np.array(V)

In [10]:
cosine_q4=V.dot(embedding_q2)
cosine_q4

array([0.92732523, 0.85948475, 0.83966021, 0.76320298, 0.76044493])

In [11]:
int(np.argmax(cosine_q4))

0

It is different.

## Q5. Selecting the embedding model

In [12]:
dim_set = set()
for model in TextEmbedding.list_supported_models():
    dim_set.add(model["dim"])
min(dim_set)

384

## Q6. Indexing with qdrant 

In [13]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [17]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

In [38]:
collection_name = "ml-zoomcamp-faq"
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

# Create the collection with specified sparse vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [39]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [44]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
query = 'I just discovered the course. Can I join now?'

In [42]:
def vector_search(question):    
    query_points = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point)
    
    return results

In [43]:
vector_search(query)[0].score

0.8703172